In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt  
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
from sklearn.metrics import r2_score
import sys
sys.path.append("../../utils")
import utils
import os
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.model_selection import cross_val_score

In [3]:
df = pd.read_excel("../../../site_info_ver_3.2.xlsx")

In [4]:
res = pd.read_excel("../../../POLY_RESULTS.xlsx")

In [6]:
df.reset_index(drop=True,inplace=True)
df = df.dropna(axis = 0)
dummies = pd.get_dummies(df, 'Site_Name')
dummies.drop(['Model_number','Date'] , axis = 1, inplace = True)
dummies = dummies.astype({'Total':np.float64,'Israelis_Count':np.float64, 'Tourists_Count':np.float64})
dummies.isna().any().sum()
dummies = dummies.loc[dummies.is_weekend == 1]

In [9]:
poly_res = pd.read_excel("./poly.xlsx")

weekend model : general , corr>=0.1

In [11]:
targets = ['Total' , 'Israelis_Count', 'Tourists_Count' ]
ys = [dummies.Total , dummies.Israelis_Count , dummies.Tourists_Count]


index = 0
for target in targets:
    #print(target+"\n")
    f = open("./"+target+"_general_weekend.txt" , "w")
    for i in range(1,5):
        to_drop = [x for x in targets if x != target]
        X = dummies.drop(to_drop , axis=1)
        corr_df = X.corr()
        correlated = corr_df[targets[index]].loc[(abs(corr_df[targets[index]]) >= 0.10)]
        
        correlated = correlated.drop([targets[index]]).index.tolist()
        if len(correlated) == 0:
            train_df = None
            test_df = None
            continue
        #targets = [model.Total , model.Israelis_Count , model.Tourists_Count]
        #targets_vals = ['Israelis_Count','Tourists_Count','Total']
        
        #if len(X) == 0: return None , None
        X = X[correlated]
        if len(X) == 0: 
            train_df = None
            test_df = None
            continue
        
        y = ys[index]
        #print(X)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7254)
        train_df = pd.merge(left=X_train, right=y_train, left_index=True, right_index=True)
        test_df = pd.merge(left=X_test, right=y_test, left_index=True, right_index=True)
        x_train_scaler = MinMaxScaler()
        x_test_scaler = MinMaxScaler()
        # y_train_scaler = MinMaxScaler()
        # y_test_scaler = MinMaxScaler()
        X_train_scaled = x_train_scaler.fit_transform(X_train)
        X_test_scaled = x_test_scaler.fit_transform(X_test)
        # y_train_scaled = y_train_scaler.fit_transform(pd.DataFrame(y_train))
        # y_test_scaled = y_test_scaler.fit_transform(pd.DataFrame(y_test))
        y_train_scaled = np.log(y_train + 0.01)
        y_test_scaled = np.log(y_test + 0.01)
        poly = PolynomialFeatures(degree=i)
        
        #fit the x variable to fit a 2rd degree polynomial value
        X_poly = poly.fit_transform(X_train_scaled)
        poly.fit(X_poly, y_train_scaled)
        pol_lin_reg = LinearRegression()
        
        pol_lin_reg.fit(X_poly, y_train_scaled)
        
        #predict the training data
        y_train_pred_scaled = pol_lin_reg.predict(poly.fit_transform(X_train_scaled))
        #y_train_pred = y_train_scaler.inverse_transform(y_train_pred_scaled)
        y_train_pred = np.exp(y_train_pred_scaled)
        #create a pandas series of the results
        y_train_pred = round(pd.Series(y_train_pred, index=y_train.index, name='predicted_entries_train'),ndigits=2)
        
        #Add the results to the DF
        train_df = pd.merge(left=train_df, right=y_train_pred , left_index=True, right_index=True)
        #train_df.head()
        y_test_pred_scaled = pol_lin_reg.predict(poly.fit_transform(X_test_scaled))
        y_test_pred = np.exp(y_test_pred_scaled)
        #y_test_pred = y_test_scaler.inverse_transform(y_test_pred_scaled)
        #create a pandas series of the results
        y_test_pred = round(pd.Series(y_test_pred, index=y_test.index, name='predicted_entries_test'),ndigits=2)
        #Add the results to the DF
        test_df = pd.merge(left=test_df, right=y_test_pred , left_index=True, right_index=True)
        if train_df is None and test_df is None : continue
        train_mse = metrics.mean_squared_error(train_df[targets[index]], train_df.predicted_entries_train)
        train_rmse = np.sqrt(metrics.mean_squared_error(train_df[targets[index]], train_df.predicted_entries_train))
        train_mae = metrics.mean_absolute_error(train_df[targets[index]], train_df.predicted_entries_train)
        train_std = np.std(train_df.predicted_entries_train)
        r2_train = r2_score(train_df[targets[index]], train_df.predicted_entries_train)
        f.write("Degrees: "+str(i)+",Target: "+target+"\n")
        f.write("------ TRAIN DATA ------\n")
        f.write("MSE : "+str(train_mse)+", RMSE: "+str(train_rmse)+", MAE : "+str(train_mae)+"\n")
        f.write("R2 TRAIN "+ str(r2_train)+"\n")
        f.write("TRAIN STD "+str(train_std)+"\n")
        try:
            test_mse = metrics.mean_squared_error(test_df[targets[index]], test_df.predicted_entries_test)
        except: pass
        try:
            test_rmse = np.sqrt(metrics.mean_squared_error(test_df[targets[index]], test_df.predicted_entries_test))
        except: pass
        try:
            test_mae = metrics.mean_absolute_error(test_df[targets[index]], test_df.predicted_entries_test)
        except: pass
        try:
            r2_test = r2_score(test_df[targets[index]], test_df.predicted_entries_test)
        except: pass
        
        test_std = np.std(test_df.predicted_entries_test)
        f.write("------ TEST DATA ------\n")
        f.write("MSE : "+str(test_mse)+", RMSE: "+str(test_rmse)+", MAE : "+str(test_mae)+"\n")
        f.write("R2 TEST "+ str(r2_test)+"\n")
        f.write("TEST STD "+str(test_std)+"\n")
        f.write("--------------------------------\n")
        
        coeff = pol_lin_reg.coef_
        #print(coeff)
        #intercept = pol_lin_reg.intercept_
        equation =""+ str(pol_lin_reg.intercept_)
        for idx in range(len(X.columns)):
            equation += " + "+str(round(coeff[idx],4))+" * "+str(X.columns[idx])
        new_row = {'Descripton':'general weekend model , corr >= 0.10 ,','Degree':str(i),'Target':target,'Model_type':'POLY',
        'MAE_Training':train_mae,'MSE_Training':train_mse,
    'RMSE_Training': train_rmse,'R2_Training': r2_train,
    'MAE_Test':test_mae,'MSE_Test':test_mse,'RMSE_Test': test_rmse,'R2_Test': r2_test,'TRAIN_STD':train_std,
    'TEST_STD':test_std , 'EQUATION' : str(equation),'TEST_SIZE' : len(test_df),'TRAIN_SIZE':len(train_df)}
    
        poly_res = poly_res.append(new_row,ignore_index=True)
        #return train_df,test_df
    index += 1
    f.close()
poly_res.to_excel("./poly.xlsx",index= False)

weekend model : general , uni test

In [ ]:
targets = ['Total' , 'Israelis_Count', 'Tourists_Count' ]
ys = [dummies.Total , dummies.Israelis_Count , dummies.Tourists_Count]


index = 0
for target in targets:
    #print(target+"\n")
    f = open("./"+target+"_weekend_UNI.txt" , "w")
    to_drop = [x for x in targets if x != target]
    #print(to_drop)
    for i in range(1,5):
        
        X = dummies.drop(to_drop , axis=1)
        
        y = ys[index]
        chi_selector = SelectKBest(f_regression, k=10)
        chi_selector.fit(X, y)
        chi_support = chi_selector.get_support()
        chi_feature = X.loc[:,chi_support].columns.to_list()
        if target in chi_feature : chi_feature.remove(target)
        X = X[chi_feature]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7254)
        train_df = pd.merge(left=X_train, right=y_train, left_index=True, right_index=True)
        test_df = pd.merge(left=X_test, right=y_test, left_index=True, right_index=True)
        x_train_scaler = MinMaxScaler()
        x_test_scaler = MinMaxScaler()
        # y_train_scaler = MinMaxScaler()
        # y_test_scaler = MinMaxScaler()
        X_train_scaled = x_train_scaler.fit_transform(X_train)
        X_test_scaled = x_test_scaler.fit_transform(X_test)
        # y_train_scaled = y_train_scaler.fit_transform(pd.DataFrame(y_train))
        # y_test_scaled = y_test_scaler.fit_transform(pd.DataFrame(y_test))
        y_train_scaled = np.log(y_train + 0.01)
        y_test_scaled = np.log(y_test + 0.01)
        poly = PolynomialFeatures(degree=i)
        
        #fit the x variable to fit a 2rd degree polynomial value
        X_poly = poly.fit_transform(X_train_scaled)
        poly.fit(X_poly, y_train_scaled)
        pol_lin_reg = LinearRegression()
        
        pol_lin_reg.fit(X_poly, y_train_scaled)
        
        #predict the training data
        y_train_pred_scaled = pol_lin_reg.predict(poly.fit_transform(X_train_scaled))
        #y_train_pred = y_train_scaler.inverse_transform(y_train_pred_scaled)
        y_train_pred = np.exp(y_train_pred_scaled)
        #create a pandas series of the results
        y_train_pred = round(pd.Series(y_train_pred, index=y_train.index, name='predicted_entries_train'),ndigits=2)
        
        #Add the results to the DF
        train_df = pd.merge(left=train_df, right=y_train_pred , left_index=True, right_index=True)
        #print(train_df.columns)
        y_test_pred_scaled = pol_lin_reg.predict(poly.fit_transform(X_test_scaled))
        y_test_pred = np.exp(y_test_pred_scaled)
        #y_test_pred = y_test_scaler.inverse_transform(y_test_pred_scaled)
        #create a pandas series of the results
        y_test_pred = round(pd.Series(y_test_pred, index=y_test.index, name='predicted_entries_test'),ndigits=2)
        #Add the results to the DF
        test_df = pd.merge(left=test_df, right=y_test_pred , left_index=True, right_index=True)
        if train_df is None and test_df is None : continue
        train_mse = metrics.mean_squared_error(train_df[targets[index]], train_df.predicted_entries_train)
        train_rmse = np.sqrt(metrics.mean_squared_error(train_df[targets[index]], train_df.predicted_entries_train))
        train_mae = metrics.mean_absolute_error(train_df[targets[index]], train_df.predicted_entries_train)
        train_std = np.std(train_df.predicted_entries_train)
        r2_train = r2_score(train_df[targets[index]], train_df.predicted_entries_train)
        f.write("Degrees: "+str(i)+",Target: "+target+"\n")
        f.write("------ TRAIN DATA ------\n")
        f.write("MSE : "+str(train_mse)+", RMSE: "+str(train_rmse)+", MAE : "+str(train_mae)+"\n")
        f.write("R2 TRAIN "+ str(r2_train)+"\n")
        f.write("TRAIN STD "+str(train_std)+"\n")
        try:
            test_mse = metrics.mean_squared_error(test_df[targets[index]], test_df.predicted_entries_test)
        except: pass
        try:
            test_rmse = np.sqrt(metrics.mean_squared_error(test_df[targets[index]], test_df.predicted_entries_test))
        except: pass
        try:
            test_mae = metrics.mean_absolute_error(test_df[targets[index]], test_df.predicted_entries_test)
        except: pass
        try:
            r2_test = r2_score(test_df[targets[index]], test_df.predicted_entries_test)
        except: pass
        
        test_std = np.std(test_df.predicted_entries_test)
        f.write("------ TEST DATA ------\n")
        f.write("MSE : "+str(test_mse)+", RMSE: "+str(test_rmse)+", MAE : "+str(test_mae)+"\n")
        f.write("R2 TEST "+ str(r2_test)+"\n")
        f.write("TEST STD "+str(test_std)+"\n")
        f.write("--------------------------------\n")
        
        coeff = pol_lin_reg.coef_
        #print(coeff)
        #intercept = pol_lin_reg.intercept_
        equation =""+ str(pol_lin_reg.intercept_)
        for idx in range(len(X.columns)):
            equation += " + "+str(round(coeff[idx],4))+" * "+str(X.columns[idx])
        new_row = {'Descripton':'weekend model ,Uni var feature selection, k=10','Degree':str(i),'Target':target,'Model_type':'POLY',
        'Degree' : str(i),'MAE_Training':train_mae,'MSE_Training':train_mse,
    'RMSE_Training': train_rmse,'R2_Training': r2_train,
    'MAE_Test':test_mae,'MSE_Test':test_mse,'RMSE_Test': test_rmse,'R2_Test': r2_test,'TRAIN_STD':train_std,
    'TEST_STD':test_std , 'EQUATION' : str(equation),'TEST_SIZE' : len(test_df),'TRAIN_SIZE':len(train_df)}
    
        poly_res = poly_res.append(new_row,ignore_index=True)
        #return train_df,test_df
    index += 1
    f.close()
poly_res.to_excel("./poly.xlsx",index= False)

weekend model: per site